<a href="https://colab.research.google.com/github/SenaKymk/HSenaKymk_YZM212_lab_proje/blob/main/README.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Özdeğerler, Özvektörler**
# *Kavramsal Tanımlar*
**Özvektör (Eigenvector):**
 Bir matrisle çarpıldığında sadece skaler bir katsayı kadar yönü değişmeyen vektörlerdir.

**Özdeğer (Eigenvalue):**

 Bu skaler katsayının kendisidir.

 Denklem: A * x = λ * x

Burada A kare matris, x bir özvektör, λ ise karşılık gelen özdeğerdir.

# **2. Makine Öğrenmesindeki Kullanımlar:**

## 1. PCA (Principal Component Analysis)
- Çok boyutlu verilerin boyutunu azaltmak için kullanılır.
- Kovaryans matrisinin özdeğerleri ve özvektörleri ile çalışılır.
- Özvektörler: Yeni eksen yönlerini belirtir.
- Özdeğerler: Bu eksenlerin önem derecesini gösterir.
- Veri, en büyük özdeğere karşılık gelen eksenler boyunca projeksiyona uğratılır.

## 2. SVD (Singular Value Decomposition)
- Herhangi bir boyutlardaki matrisi 3 bileşene ayırır:  
  $$ A = U \cdot \Sigma \cdot V^T $$
- \( U \): Orijinal uzaydaki ortonormal sütun vektörlerini içerir (sol özvektörler).
- \( \Sigma \): Diagonal matristir, pozitif ve azalan sırada sıralı singular değerler içerir.
- \( V^T \): Özvektörlerin transpozu, çıkış uzayındaki yönleri temsil eder.
- Sıkıştırma, gürültü azaltma ve boyut indirgemede etkilidir.

## 3. Google PageRank
- İnternetteki sayfalar arası geçişleri modelleyen büyük olasılık matrisleri ile çalışır.
- Geçiş matrisi üzerinde dominant özvektör hesaplanır.
- Bu özvektör, her sayfanın "önem skorunu" belirler.
- Temel denklem: $$ P \cdot x = x $$
- Burada \( x \), PageRank skor vektörüdür.

## 4. Spectral Clustering
- Veriler arası benzerlik matrisi oluşturulur (örneğin Gaussian kernel ile).
- Bu matrisin özdeğer ve özvektörleri hesaplanır.
- İlk k adet özvektör, yeni öznitelik uzayı olarak kullanılır.
- K-means gibi yöntemlerle bu yeni uzayda kümeleme yapılır.


### Karakteristik polinom:
$$ |A - \lambda I| = 0 $$  
- Bu denklem özdeğerleri verir, ardından özvektörler çözülür.

### Eigen Decomposition
- Genel form: $$ A = X \Lambda X^{-1} $$
- Simetrik matrislerde: $$ A = Q \Lambda Q^T $$ (Q ortogonal matristir)

### SVD:
- Genel form: $$ A = U \Sigma V^T $$
- Boyut indirgeme, bilgi kaybını minimize ederek veri sıkıştırmada kullanılır.

### Vektör Normu:
- $$ ||x|| = \sqrt{x^T x} $$

# **3. NumPy `linalg.eig` Fonksiyonunu İnceleme:**

NumPy'nin `linalg.eig` fonksiyonu, kare matrislerin özdeğerlerini ve özvektörlerini hesaplamada kullanılan güçlü bir lineer cebir aracıdır. Bu inceleme, hem fonksiyonun kullanıcı düzeyinde nasıl çalıştığını hem de `numpy/numpy` GitHub deposundaki kaynak kodlar üzerinden arka plandaki detayları kapsamlı biçimde açıklamaktadır.

---

## 📌 Fonksiyon Tanımı ve Kullanımı
```python
numpy.linalg.eig(a)
```
- Bu fonksiyon kare bir matrisin (veya matris yığınının) özdeğerlerini ve sağ özvektörlerini döndürür.

| Parametre | Açıklama |
|-----------|----------|
| `a`       | (..., M, M) boyutlarında bir kare matris veya matris yığını |

| Dönüş Değeri  | Açıklama |
|----------------|----------|
| `eigenvalues`  | (..., M) boyutunda — özdeğerleri içerir |
| `eigenvectors` | (..., M, M) boyutunda — sütunları normalleştirilmiş özvektörlerdir |

🔸 **Not:** Yakınsama olmazsa `LinAlgError` hatası yükselir.

---

## ⚙️ Python Seviyesi İncelemesi (`_linalg.py`)
NumPy'nin `linalg.eig` fonksiyonu, üst düzey Python fonksiyonlarının bir sarmalayıcısıdır. Gerçek işlem `umath_linalg` modülüne paslanmadan önce bazı kontroller yapılır:

| Adım | Açıklama |
|------|----------|
| 1 | `a, wrap = _makearray(a)` — Girdiyi ndarray formatına çevirir |
| 2 | `_assert_stacked_2d(a)` — Matrisin en az 2 boyutlu olduğunu kontrol eder |
| 3 | `_assert_stacked_square(a)` — Kare matris olup olmadığını kontrol eder |
| 4 | `t, result_t = _commonType(a)` — Veri türünü belirler (float/complex) |
| 5 | `signature = 'D->D'` veya `'d->d'` — uygun LAPACK fonksiyonunun seçilmesini sağlar |
| 6 | `w, v = _umath_linalg.eig(...)` — Asıl hesaplama çağrısı yapılır |
| 7 | `wrap(EigResult(...))` — Sonuçları uygun yapıya sokar ve döndürür |

Bu adımlar Python seviyesinde giriş verisinin düzgün işlenmesini ve doğru LAPACK çağrısının yapılmasını garanti eder.

---

##  C/C++ Seviyesi İncelemesi (`umath_linalg.c.src`)

Düşük seviyeli hesaplamalar `umath_linalg.c.src` içinde gerçekleşir. `eig` fonksiyonu şu şekilde tanımlanır:
```c
{"eig", "(m,m)->(m),(m,m)", FUNC_ARRAY_NAME(eig), eig_types, ...}
```
Bu satırda belirtilen yapı, fonksiyonun giriş ve çıkış şekillerini tanımlar:
- Girdi: (m, m) kare matris
- Çıktı: (m) özdeğerler, (m, m) özvektörler

### 🧩 İşlem Adımları:
1. `linearize_matrix` fonksiyonu ile giriş matrisi düz belleğe (tek boyutlu) çevrilir.
2. `dgeev` (gerçek sayılar için) veya `zgeev` (karmaşık sayılar için) LAPACK fonksiyonu çağrılır.
3. Hesaplama tamamlandıktan sonra `delinearize_matrix` fonksiyonu ile veriler matris formuna geri dönüştürülür.

### 🔄 Kullanılan LAPACK Fonksiyonları:
| Fonksiyon | Görevi |
|-----------|--------|
| `dgeev`   | double-precision gerçek matrisler için eigen çözümü |
| `zgeev`   | complex matrisler için çözüm |
| `FUNC_ARRAY_NAME(eig)` | doğru LAPACK fonksiyonunu seçmek için kullanılır |

Bu C düzeyi yapı, Python kullanıcıları için son derece hızlı ve bellek verimli özdeğer hesaplamaları sağlar.

---

##  Kaynak Dosya Hiyerarşisi (GitHub Reposuna Göre)
GitHub'daki `numpy/numpy/tree/main/numpy/linalg` dizininde yer alan önemli dosya ve klasörler:

| Dosya/Klasör | Açıklama |
|--------------|----------|
| `linalg.py`  | Kullanıcıya sunulan API fonksiyonlarını içerir |
| `_linalg.py` | Giriş kontrolü, hata yönetimi ve tür eşlemesi gibi işlemleri yapar |
| `umath_linalg.c.src` | LAPACK çağrılarını içeren düşük seviyeli C fonksiyonlarını barındırır |
| `tests/`     | Fonksiyonların doğruluğunu test eden birim testleri içerir |

---

## ✅ Sonuç ve Değerlendirme
- NumPy `linalg.eig` fonksiyonu, arka planda karmaşık bir işleme sürecini kullanıcıdan gizleyerek basit bir API sunar.
- Python düzeyinde esneklik ve hata kontrolü sağlanırken, C/C++ düzeyinde LAPACK tabanlı yüksek performanslı lineer cebir işlemleri gerçekleştirilir.
- Gerek akademik çalışmalar gerekse büyük veri analitiği işlemleri için güvenilir bir çözümdür.

---




# **4. EigenVectorsValues Dosyası İnceleme:**

Bu proje kapsamında, EigenVectorsValues.ipynb adlı dosyada NumPy'in linalg.eig fonksiyonu kullanılmadan, manuel olarak özdeğerlerin nasıl hesaplandığı gösterilmiştir. Bu uygulamada:

- 2x2 boyutunda bir matris için karakteristik denklem kurulmuş,

- Polinom katsayıları elde edilerek numpy.roots() fonksiyonu ile kökler yani özdeğerler hesaplanmıştır,

- Aynı matris için numpy.linalg.eig fonksiyonu kullanılarak özdeğer ve özvektörler hesaplanmış ve iki yöntem karşılaştırılmıştır.



## ✅ Sonuç ve Değerlendirme
Manuel ve hazır fonksiyonlar kullanılarak yapılan analizler sonucunda:

Her iki yöntemle elde edilen özdeğerler tamamen uyuşmuştur.

NumPy'in eig fonksiyonu ile elde edilen özvektörler, doğrusal denklem

𝐴
𝑐
𝑑
𝑜
𝑡
𝑣
=
𝑙
𝑎
𝑚
𝑏
𝑑
𝑎
𝑐
𝑑
𝑜
𝑡
𝑣
A
cdotv=
lambda
cdotv
eşitliğini sağlamaktadır.

np.allclose() fonksiyonu ile yapılan sayısal karşılaştırmalar doğrultusunda, sonuçların sayısal olarak eşdeğer olduğu gözlemlenmiştir.

Ayrıca özvektörler vektör alanında görselleştirilmiş ve doğrultularının doğru olduğu grafikle gösterilmiştir.

*🎯 Sonuç Olarak*

Bu çalışma, özdeğerlerin manuel ve hazır fonksiyonlarla nasıl hesaplandığını ve bu iki yöntemle elde edilen sonuçların birbiriyle nasıl karşılaştırılabileceğini göstermektedir.

Manuel yöntem, temel matematiksel kavrayış açısından eğiticidir.

NumPy linalg.eig fonksiyonu ise pratik, hızlı ve güvenilirdir; bilimsel hesaplamalarda doğrudan tercih edilebilir.

## 🔗 Kaynakça

- NumPy `eig` Fonksiyonu Dokümantasyonu:  
  https://numpy.org/doc/stable/reference/generated/numpy.linalg.eig.html

- NumPy `linalg` GitHub Kodu:  
  https://github.com/numpy/numpy/tree/main/numpy/linalg

- LucasBN GitHub - Manuel Özdeğer Hesaplama:  
  https://github.com/LucasBN/Eigenvalues-and-Eigenvectors

- Jason Brownlee – Machine Learning Mastery:  
  https://machinelearningmastery.com/introduction-to-eigendecomposition-eigenvalues-and-eigenvectors/

- Burak Tahtacıoğlu - Özvektör ve Özdeğer Blog Yazısı (TR):  
  https://buraktahtacioglu.blogspot.com/2019/08/veri-bilimi-icin-lineer-cebir-linear.html

- Matematik Dünyası – Özdeğer ve Özvektörler (TR):  
  https://www.matematikdunyasi.org/2014/02/ozdegerler-ve-ozvektorler.html